In [5]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# !pip install -U dash

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/7b/62/438626ab498869249e99376684ba6d88931e7c2dd86d03a92d9d7a9b4c54/dash-2.14.2-py3-none-any.whl.metadata
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/10.2 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.1/10.2 MB 573.4 kB/s eta 0:00:18
   ---------------------------------------- 0.1/10.2 MB 731.4 kB/s eta 0:00:14
    --------------------------------------- 0.2/10.2 MB 748.1 kB/s eta 0:00:14
    --------------------------------------- 0.2/10.2 MB 1.0 MB/s eta 0:00

Defaulting to user installation because normal site-packages is not writeable


In [7]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [8]:
launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
all_launch_sites = spacex_df['Launch Site'].unique().tolist()
for launch_site in all_launch_sites:
    launch_sites.append({'label': launch_site, 'value': launch_site})

In [13]:
app = dash.Dash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 50}),
                                html.Div([
                                    dcc.Dropdown(
                                        id = 'site-dropdown',
                                        options = launch_sites,
                                        placeholder = 'Select a Launch Site here',
                                        searchable = True ,
                                        clearable = False,
                                        value = 'All Sites'
                                        ),]),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.Br(),
                                html.P("Payload range (Kg):", style={'font-size':20}),
                                html.Div([
                                    dcc.RangeSlider(
                                        id = 'payload_slider',
                                        min = 0,
                                        max = 10000,
                                        step = 1000,
                                        marks = {
                                            0: {'label': '0 Kg', 'style': {'color': '#77b0b1'}},
                                            1000: {'label': '1000 Kg'},
                                            2000: {'label': '2000 Kg'},
                                            3000: {'label': '3000 Kg'},
                                            4000: {'label': '4000 Kg'},
                                            5000: {'label': '5000 Kg'},
                                            6000: {'label': '6000 Kg'},
                                            7000: {'label': '7000 Kg'},
                                            8000: {'label': '8000 Kg'},
                                            9000: {'label': '9000 Kg'},
                                            10000: {'label': '10000 Kg'},
                                            },
                                            value = [min_payload,max_payload]),
                                            ]),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(
     Output(component_id = 'success-pie-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value')]
)
def update_piegraph(site_dropdown):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        data  = spacex_df[spacex_df['class'] == 1] 
        fig = px.pie(
                data,
                names = 'Launch Site',
                title = 'Total Success Launches by All Sites',
            )
    else:
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(
                data,
                names = 'class',
                title = 'Total Success Launches for Site: '+site_dropdown,
            )
    return fig

@app.callback(
     Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value'), 
     Input(component_id = "payload_slider", component_property = "value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        low, high = payload_slider
        data  = spacex_df
        inrange = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(
                data[inrange], 
                x = "Payload Mass (kg)", 
                y = "class",
                title = 'Correlation Between Payload and Success for All Sites',
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    else:
        low, high = payload_slider
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        inrange = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(
                data[inrange],
                x = "Payload Mass (kg)",
                y = "class",
                title = 'Correlation Between Payload and Success for Site: '+site_dropdown,
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    return fig

if __name__ == '__main__':
    app.run_server()